Lab03
=======

### Context
#### Dimension Reduction
+ PCA

# 차원 축소가 필요한 이유 - 차원의 저주
차원의 저주는 저차원에서는 일어나지 않는 현상들이 고차원에서 데이터를 분석하거나 다룰 때 생겨나는 현상을 말합니다.<br>
고차원으로 올라갈 수록 공간의 크기가 증가하게 되는데, 데이터는 해당 공간에 한정적으로 위치되어 빈 공간이 많아지기 때문에 발생합니다.<br>
이러한 이유로 데이터의 차원이 너무 큰 경우에는 필요없는 변수를 제거하고, 과적합을 방지하기위해 데이터의 차원을 축소합니다. <br>
또는, 사람이 인식할 수 있는 차원은 3차원이 최대이므로 데이터의 시각화를 위해 차원을 축소하기도 합니다.

![CurseofDimensionality](./Images/Curse_of_Dimensionality.png)


## 주 성분 분석 (Principal Component Analysis, PCA)
대표적인 차원 축소 기법으로 주 성분 분석(이하, PCA)이라는 방법이 있습니다.<br>
PCA는 여러 차원으로 이루어진 데이터를 가장 잘 표현하는 축으로 Projection 해서 차원을 축소하는 방식을 사용합니다.<br>
데이터를 가장 잘 표현하는 축이란, 데이터의 분산을 잘 표현하는 축이라고 할 수 있습니다.<br>
기본적으로 주성분(Principal Component, PC)은 데이터 셋을 특이값 분해를 통해 추출된 고유 벡터입니다.<br>
각 고유 벡터들은 서로 직교성을 띄기 때문에 데이터를 주성분로 Projection 시켰을 때 서로 독립적으로 데이터를 잘 표현할 수 있습니다.<br>
PCA의 단점으로는 떨어뜨린 주성분이 어떤 컬럼인지를 설명할 수 없다는 점이 있습니다. 

#### 주 성분 분석의 단계
1. 각 컬럼들의 값의 범위를 평균과 표준편차를 사용해 정규화시켜 동일하게 만들어줍니다.
2. 데이터의 공분산을 계산합니다.
3. 공분산 행렬에 대해 특이값 분해를 하여 주성분(고유 벡터)과 고유 값을 얻어냅니다.
4. 주성분과 대응되는 고유 값은 주성분이 데이터의 분산을 표현하는 정도의 척도로 사용되므로, 고유 값의 크기와 비율을 보고 몇개의 주성분을 선택할 것인지 또는 원하는 차원의 개수만큼의 주성분을 선택합니다.
5. 선택한 주성분으로 모든 데이터를 Projection시켜 데이터의 차원을 축소합니다.

#### Projection(사영)
Projection에 대해 간단히 짚고 넘어가겠습니다. <br>
벡터 공간에서 어떤 벡터 a와 b가 있을 때 벡터 b를 벡터 a에 사영한 결과는 아래 그림과 같습니다.<br>
벡터 b를 벡터 a에 사영한다는 것은 벡터 a에 대해 수직인 방향으로 벡터 b를 곧 바로 떨어뜨리는 것을 의미합니다.<br>
간단히 말해서, 벡터 b의 그림자를 벡터 a에 떨어뜨린 것을 생각하시면 편합니다.
![Projection](./Images/Projection.png)
PCA의 기본 원리는 데이터의 분산을 가장 잘 표현하는 벡터를 찾아 해당 벡터에 데이터들을 사영 시키는 것입니다.

In [ ]:
import sys
from os.path import join
import matplotlib
import numpy as np
import pylab as plt
import matplotlib.cm as cm
import matplotlib.image as mpimg
from sklearn.decomposition import PCA



## 1. 데이터 살펴보기
이번에 사용할 데이터는 손글씨 데이터로 유명한 MNIST 데이터 입니다.<br>
실습의 편의를 위해 데이터는 28*28 행렬로 변환되어 저장된 상태입니다.<br>
data 디렉토리에 준비된 데이터와 라벨을 가져옵니다.

In [ ]:
X = np.load(join('data','MNIST.npy'))
labels = np.load(join('data','Label.npy'))

In [ ]:
img=plt.imshow(X[1].reshape((28,28)))

In [ ]:
labels[1]

1번 인덱스에 있던 값은 0에 해당하는 손글씨 데이터입니다.<br>
라벨도 확인해보니 0이 맞습니다.

## 2. 간단한 전처리
MNIST 데이터는 28*28에 픽셀마다 0~255 범위의 값을 가진 행렬입니다. 
각 픽셀에 대해 0~1 사이의 값을 갖도록 255로 나누어줍니다.
그리고 네트워크에 데이터를 입력하기 편하게 하도록 2차원 이미지를 벡터형태로 펼치겠습니다.
numpy.reshape() 함수를 통해 행렬의 차원을 변경할 수 있습니다. 

In [ ]:
X = X/255
X = X.reshape(len(X),784)

값의 범위가 0과 1사이로 잘 되었는지 확인해봅시다.
numpy.max(), numpy.min() 으로 확인해 볼 수 있습니다.

In [ ]:
print('최대 : {}, 최소 : {}'.format(np.max(X), np.min(X)))

## 3. Scratch로 PCA 해보기
이미지 자체도 28*28 = 784 차원의 데이터라고 생각할 수 있습니다.<br>
2차원 이상의 데이터는 우리가 점으로 표현하기 어려우니 784차원 데이터를 2차원으로 축소시켜 보겠습니다.

In [ ]:
X = np.load(join('data','MNIST.npy'))[:2500]

labels = np.load(join('data','Label.npy'))[:2500]

In [ ]:
X = X.reshape((2500,784))

In [ ]:
no_dims = 2
# '2.간단한 전처리'에서 각 픽셀의 값의 범위를 0~1 사이로 정규화를 수행했으니 추가적인 정규화를 수행하지 않아도 됩니다.

#공분산
cov = np.cov(X.T) #(784,784)

#특이값분해
u,d,v = np.linalg.svd(cov)

#no_dims 만큼의 고유 벡터 취하기
uk = u[: , :no_dims]
#(784, no_dims)

#no_dims만큼의 고유 벡터에 모든 x를 사영 uk = (784, no_dims) X = (784, 1000)
#uk를 Transpose하고 내적하면 uk.T = (no_dims,784) * X = (784, 1000) -> z.T = (1000, no_dims)
z = np.dot(X,uk)

In [ ]:
def plotting(Y, labels):
    legend_ = []; colors = cm.rainbow(np.linspace(0, 1, 10))
    for i in sorted(list(set(labels))):
        idxs = (labels==i).nonzero()
        l = plt.scatter(np.squeeze(Y[idxs,0]), Y[idxs,1], 20, color=colors[int(i)])
        legend_.append(l)
    plt.legend(legend_, list(range(10)), loc='center left', ncol=1, fontsize=8, bbox_to_anchor=(1, 0.5))
    return

In [ ]:
plotting(z, labels)

## 4. Sklearn으로 PCA 해보기
파이썬 코드로 PCA의 단계별로 살펴보았습니다.<br>
하지만, Sklearn에 PCA를 쉽게 사용할 수 있도록 준비되어 있는데요.<br>
이미 기억하고 계실지 모르시겠지만, 이전의 예제에서 Sklearn의 PCA를 사용해 데이터들의 차원을 축소했었습니다.<br>
그럼 시작해보겠습니다.

In [ ]:
# PCA 모델에 축소하고자 하는 차원을 n_compoenets 값에 넣어 생성합니다.
pca = PCA(n_components=2)
# X데이터에 대해 주성분을 뽑아내야하니 X데이터 전체를 넣습니다.
pca.fit(X)
# 이제 PCA 모델이 X데이터의 주성분을 찾아냈습니다. 찾은 2개의 주성분에 X데이터를 Projection 합니다.
X = pca.transform(X)

In [ ]:
plotting(X, labels)

Sklearn을 사용하시면 더욱 쉽게 PCA를 실행할 수 있습니다.<br> 
여기에서 PCA를 보내기는 아쉬우니 고유 값을 보고 주성분의 개수를 선택하는 실습을 진행해보겠습니다.

## 5. 연습문제 : 주성분 개수 선택해보기
'3.Scratch로 PCA 해보기'에서 진행했던 Scratch 코드를 다시 살펴보겠습니다.<br>
공분산을 특이값분해 하여 얻은 u,d,v 중 u가 주성분(고유 벡터)을, d 가 고유 값을 담고 있는 행렬입니다.<br>
특이값 분해의 특징은 고유 값의 크기의 내림차순으로 정렬되어 성분들이 분해된다는 점이 있습니다.<br>
이를 이용하여 784 차원의 기존 이미지의 80% 이상의 데이터 분산을 표현할 수 있는 주성분의 개수를 찾아보세요.<br>

* 고유 값의 크기는 해당 주성분(고유 벡터)이 기존 데이터의 분산을 표현하는 정도의 척도로 사용됩니다.

In [ ]:
X = np.load(join('data','MNIST.npy'))[:2500]
X = X.reshape((2500,784))

In [ ]:
'''
고유 값 행렬 d를 이용해 80% 이상의 데이터 분산을 표혈할 수 있는 주성분 개수를 찾아보세요.
힌트 반복문을 이용하여 고유 값을 하나씩 더해보고 전체 고유 값 비율의 0.8 이상이 되었을 때의 주성분 개수를 반환하면 됩니다.
'''

#공분산
cov = np.cov(X.T)

#특이값분해
u,d,v = np.linalg.svd(cov)

print(np.shape(d))

### Reference

- 차원의 저주 - https://wikidocs.net/7646<br>
- 주성분 분석 - https://ko.wikipedia.org/wiki/주성분_분석 